## Build STAC catalog of RETREAT data

RETREAT ice velocity data generated from S1 image pairs

paper - https://essd.copernicus.org/articles/13/4653/2021/essd-13-4653-2021-discussion.html
SI - https://essd.copernicus.org/articles/13/4653/2021/essd-13-4653-2021-supplement.pdf
file download info: {insert filelist_velocitiesXXXX.txt}
data downloaded from: https://retreat.geographie.uni-erlangen.de


Used this [tutorial/example](https://towardsdatascience.com/organizing-geospatial-data-with-spatio-temporal-assets-catalogs-stac-using-python-45f1a64ca082) heavily while organizing this data as a STAC catalog.

In [81]:
from pathlib import Path
import stackstac
import xarray as xr
import glob
import shutil
import pystac
from itertools import groupby
from shapely.geometry import Polygon, mapping
import rasterio as rio
import os
import pandas as pd
import rasterio
from rasterio.warp import transform_bounds, transform_geom
from pystac.extensions import item_assets 
from pystac.extensions.projection import AssetProjectionExtension
import pandas as pd
from rasterio.warp import transform_bounds, transform_geom


In [107]:
def parse_fname(fposix):
    '''extract relevant metadata from filename and return it as variables that will be used when creating stac items
    note function takes `pathlib.PosixPath` objects, not strs
    '''
    fname = fposix.stem
    prevar = fname.split('-')[0]
    var = fname.split('-')[1].split('+')[0]
    post_var = fname.split('-')[1].split('+')[1]
    
    sensor = post_var[:2]
    acq_date = post_var[3:18] #tried formating these as datetime at first but catalog won't save if property objects are datetimes, it seems 
    acq_id = post_var[19:23]
    
    sec_date = post_var[39:54]
    sec_id = post_var[55:59]
    
    site = prevar[:2]
    frame = prevar[3:6]
    orbit = prevar[7:10]
    mid_date = pd.to_datetime(prevar[11:19])
    
    return sensor, acq_date, acq_id, sec_date, sec_id, site, orbit, frame, mid_date

In [86]:
#Create catalog
catalog = pystac.Catalog(
    id = 'RETREAT data, Langtang',
    description = 'RETREAT velocity data for glaciers in Langtang region of Nepal/Tibet',
    stac_extensions = ['https://stac-extensions.github.io/projection/v1.0.0/schema.json']
)

In [60]:
#set path to data
folder = Path('/home/emmamarshall/Desktop/phd_work/retreat/retreat_data_new_110/')
folder

PosixPath('/home/emmamarshall/Desktop/phd_work/retreat/retreat_data_new_110')

In [105]:
type(dis_angs[0])

pathlib.PosixPath

In [61]:
ds = xr.open_dataset(dis_angs[0], engine='rasterio').squeeze()

In [82]:
ds_rasterio = rasterio.open(dis_angs[0])

In [84]:
ds_rasterio.transform

Affine(200.0, 0.0, 199356.058,
       0.0, -200.0, 3251854.581)

In [62]:
ds.spatial_ref.attrs['GeoTransform']

'199356.058 200.0 0.0 3251854.581 0.0 -200.0'

In [108]:
#create filepath lists for each variable
dis_angs = list(folder.rglob('*dis_angle*.tif'))
dis_azs = list(folder.rglob('*dis_az*.tif'))
dis_mags = list(folder.rglob('*dis_mag*.tif'))
dis_N_angs = list(folder.rglob('*dis_N_ang*.tif'))
dis_rs = list(folder.rglob('*dis_r*.tif'))

In [87]:
def build_stac_catalog(dis_angs, dis_azs, dis_mags, dis_N_angs, dis_rs):
    
    #make list of all vars
    var_ls = [dis_angs, dis_azs, dis_mags, dis_N_angs, dis_rs]
    #ensure that the lists of vars are correctly ordered, ie calling same acquisition across vars
    zero_element = [var_ls[var][0].stem[:19] for var in range(len(var_ls))]

    if len(set(zero_element)) > 1:
        print('there is a file issue here')
    else:
        pass


    for file in range(len(dis_angs)):

        sensor, acq_date, acq_id, sec_date, sec_id, site, orbit, frame, mid_date = parse_fname(dis_angs[file])

        ds = xr.open_dataset(dis_angs[file], engine='rasterio')
        
        ds_rasterio = rasterio.open(dis_angs[file])
        transform = ds_rasterio.transform
        
        shape = [len(ds.x.data), len(ds.y.data)]

        left = ds.x.data.min()
        right = ds.x.data.max()
        bottom = ds.y.data.min()
        top = ds.y.data.max()

        bbox = [left, bottom, right, top]
        footprint = mapping(Polygon([
                [bbox[0], bbox[1]],
                [bbox[0], bbox[3]],
                [bbox[2], bbox[3]],
                [bbox[2], bbox[1]]
                ]))

        geo_bounds = transform_bounds(ds.rio.crs, 'EPSG:4326', *bbox)
        geo_footprint = transform_geom(ds.rio.crs, 'EPSG:4326', footprint)

        idx = dis_angs[file].stem[:19]
        #print(idx)
        #date = pd.to_datetime(dis_ang.stem[11:19])
        date = dis_angs[file].stem[11:19]
        tile = idx[:10]
        #print(tile)

        #make assets for each time step (each geotiff file)
        dis_ang_asset = pystac.Asset(
                href = dis_angs[file].as_posix(),
                media_type=pystac.MediaType.GEOTIFF,
                title = str(dis_angs[file].stem.split('-')[1].split('+')[0]),
            )
        dis_az_asset = pystac.Asset(
            href = dis_azs[file].as_posix(),
            media_type = pystac.MediaType.GEOTIFF,
            title = str(dis_azs[file].stem.split('-')[1].split('+')[0]),
        )
        dis_mag_asset = pystac.Asset(
            href = dis_mags[file].as_posix(),
            media_type = pystac.MediaType.GEOTIFF,
            title = str(dis_mags[file].stem.split('-')[1].split('+')[0]),
        )
        dis_N_ang_asset = pystac.Asset(
            href = dis_N_angs[file].as_posix(),
            media_type = pystac.MediaType.GEOTIFF,
            title = str(dis_N_angs[file].stem.split('-')[1].split('+')[0]),
        )
        dis_r_asset = pystac.Asset(
            href = dis_rs[file].as_posix(),
            media_type = pystac.MediaType.GEOTIFF,
            title = str(dis_rs[file].stem.split('-')[1].split('+')[0]),
        )
      
        #make item (1 item per timestep/siet)
        item = pystac.Item(
            id = idx,
            geometry= geo_footprint,
            bbox= geo_bounds,
            datetime= pd.to_datetime(date),
            stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json'],
            assets = {'dis_ang':dis_ang_asset,
                      'dis_az':dis_az_asset,
                      'dis_mag':dis_mag_asset,
                      'dis_N_ang':dis_N_ang_asset,
                      'dis_r':dis_r_asset},
            properties = dict(
                sensor = sensor,
                acq_date = acq_date,
                sec_date = sec_date,
                acq_id = acq_id,
                sec_id = sec_id,
                frame = frame, 
                orbit = orbit,
                site = site,
                )
        )
        #extend asset wtih projection extension
        asset_ext = AssetProjectionExtension.ext(item.assets['dis_ang'])
        asset_ext.epsg = ds.rio.crs.to_epsg()
        asset_ext.shape = shape
        asset_ext.bbox = bbox
        asset_ext.geometry = footprint
        #asset_ext.transform = ds.spatial_ref.attrs['GeoTransform']
        asset_ext.transform = transform
    
        asset_ext = AssetProjectionExtension.ext(item.assets['dis_az'])
        asset_ext.epsg = ds.rio.crs.to_epsg()
        asset_ext.shape = shape
        asset_ext.bbox = bbox
        asset_ext.geometry = footprint
        #asset_ext.transform = ds.spatial_ref.attrs['GeoTransform']
        asset_ext.transform = transform

        asset_ext = AssetProjectionExtension.ext(item.assets['dis_mag'])
        asset_ext.epsg = ds.rio.crs.to_epsg()
        asset_ext.shape = shape
        asset_ext.bbox = bbox
        asset_ext.geometry = footprint
        #asset_ext.transform = ds.spatial_ref.attrs['GeoTransform']
        asset_ext.transform = transform

    
        asset_ext = AssetProjectionExtension.ext(item.assets['dis_N_ang'])
        asset_ext.epsg = ds.rio.crs.to_epsg()
        asset_ext.shape = shape
        asset_ext.bbox = bbox
        asset_ext.geometry = footprint
        #asset_ext.transform = ds.spatial_ref.attrs['GeoTransform']
        asset_ext.transform = transform

    
        asset_ext = AssetProjectionExtension.ext(item.assets['dis_r'])
        asset_ext.epsg = ds.rio.crs.to_epsg()
        asset_ext.shape = shape
        asset_ext.bbox = bbox
        asset_ext.geometry = footprint
        #asset_ext.transform = ds.spatial_ref.attrs['GeoTransform']
        asset_ext.transform = transform

    
        catalog.add_item(item)

    
            

In [88]:
build_stac_catalog(dis_angs, dis_azs, dis_mags, dis_N_angs, dis_rs)

In [89]:
print(len(list(catalog.get_all_items())))

314


In [90]:
!pwd

/home/emmamarshall/Desktop/phd_work/retreat/retreat/code


In [91]:
catalog.normalize_hrefs('/home/emmamarshall/Desktop/phd_work/retreat/retreat/code/stac_catalog')

In [92]:
catalog.save()

In [93]:
!pwd

/home/emmamarshall/Desktop/phd_work/retreat/retreat/code


In [94]:
catalog

"ID: RETREAT data, Langtang"
Description: RETREAT velocity data for glaciers in Langtang region of Nepal/Tibet
https://stac-extensions.github.io/projection/v1.0.0/schema.json
ID: 10_085_053_20210113
"Bounding Box: (83.90435612095534, 27.316552327766512, 86.84688658910103, 29.3949542892891)"
Datetime: 2021-01-13 00:00:00
sensor: S1
acq_date: 20210107T122220
sec_date: 20210119T122219
acq_id: BFF8
sec_id: DDCD


## Testing 

In [95]:
import json

cat = pystac.Catalog.from_file('/home/emmamarshall/Desktop/phd_work/retreat/retreat/code/stac_catalog/catalog.json')
item = list(cat.get_items())

In [96]:
cat

"ID: RETREAT data, Langtang"
Description: RETREAT velocity data for glaciers in Langtang region of Nepal/Tibet
type: Catalog
https://stac-extensions.github.io/projection/v1.0.0/schema.json
ID: 10_085_053_20210113
"Bounding Box: [83.90435612095534, 27.316552327766512, 86.84688658910103, 29.3949542892891]"
Datetime: 2021-01-13 00:00:00+00:00
sensor: S1
acq_date: 20210107T122220
sec_date: 20210119T122219
acq_id: BFF8


In [97]:
item[-1]

ID: 10_019_085_20210720
"Bounding Box: [83.2442180150967, 27.56441954000287, 86.17114833425134, 29.71371916902088]"
Datetime: 2021-07-20 00:00:00+00:00
sensor: S1
acq_date: 20210714T001927
sec_date: 20210726T001927
acq_id: C779
sec_id: F61C
frame: 019
orbit: 085
site: 10


In [98]:
#read in all items
items = [item for item in catalog.get_items()]

In [99]:
items[0]

ID: 10_085_053_20210113
"Bounding Box: (83.90435612095534, 27.316552327766512, 86.84688658910103, 29.3949542892891)"
Datetime: 2021-01-13 00:00:00
sensor: S1
acq_date: 20210107T122220
sec_date: 20210119T122219
acq_id: BFF8
sec_id: DDCD
frame: 085
orbit: 053
site: 10


In [100]:
cube = stackstac.stack(
    items = [item.to_dict() for item in items],
    epsg = '32645')
   

In [102]:
cube = cube.rename({'band':'variable'})

In [103]:
cube

<xarray.DataArray 'stackstac-2aeace70d1b95b228bf0811f10c55af1' (time: 314,
                                                                variable: 5,
                                                                y: 2180, x: 1636)>
dask.array<fetch_raster_window, shape=(314, 5, 2180, 1636), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/16)
  * time       (time) datetime64[ns] 2014-10-23 2014-10-31 ... 2021-11-05
    id         (time) <U19 '10_085_002_20141023' ... '10_019_085_20211105'
  * variable   (variable) <U9 'dis_ang' 'dis_az' 'dis_mag' 'dis_N_ang' 'dis_r'
  * x          (x) float64 1.128e+05 1.13e+05 1.132e+05 ... 4.396e+05 4.398e+05
  * y          (y) float64 3.357e+06 3.357e+06 3.357e+06 ... 2.921e+06 2.921e+06
    sensor     <U2 'S1'
    ...         ...
    site       <U2 '10'
    sec_id     (time) <U4 '55_D' '54_1' '3910' '54_C' ... 'F61C' 'D582' '3830'
    frame      (time) <U3 '085' '019' '019' '085' ... '019' '019' '019' '019'
    title      (variable) <U9 'dis_angle' 'dis_az' 'dis_mag' 'dis_N_ang' 'dis_r'
    proj:epsg  int64 32645
    epsg       <U5 '32645'
Attributes:
    spec:        RasterSpec(epsg='32645', bounds=(112800.0, 2921000.0, 440000...
    crs:         epsg:32645
    transform:   | 200.00, 0.00, 112800.00|\n| 0.00,-200.00, 3357000.00|\n| 0...
    resolution:  200.0